## Imports

In [43]:
import random
from functools import partial

from scipy.optimize import OptimizeResult
from scipy.optimize import minimize

from src.data import *

## Simulated Annealing

In [44]:
def simulated_annealing(objective, neighbor_fn, initial_state, initial_temp=10.0, final_temp=1e-3, alpha=0.5,
                        max_iter=100):
    nfev = 0
    njev = 0
    nit = 0

    current_state = initial_state
    current_score = objective(current_state)
    nfev += 1
    best_state, best_score = current_state, current_score

    t = initial_temp

    while t > final_temp:
        accepted = 0
        for _ in range(max_iter):
            nit += 1
            candidate = neighbor_fn(current_state)
            njev += 1
            candidate_score = objective(candidate)
            nfev += 1

            delta = candidate_score - current_score
            if delta < 0 or random.random() < math.exp(-delta / t):
                current_state, current_score = candidate, candidate_score
                accepted += 1
                if current_score < best_score:
                    best_state, best_score = current_state, current_score

        if accepted == 0:
            break

        t *= alpha

    return OptimizeResult(
        x=best_state,
        fun=best_score,
        nfev=nfev,
        njev=njev,
        nit=nit,
    )

## Benchmarking

In [45]:
def run_bench(objective_func,
              initial_point=None,
              max_iter=1000,
              true_optimum_value=0.0,
              neighbor_step=0.1):
    if initial_point is None:
        initial_point = np.array([5.0, -5.0])

    results = {}

    def neighbor_fn(point, step=neighbor_step):
        return point + np.random.uniform(-step, step, size=len(point))

    sa_result = simulated_annealing(
        objective=objective_func,
        neighbor_fn=lambda p: neighbor_fn(p),
        initial_state=initial_point,
        initial_temp=30.0,
        final_temp=1e-5,
        alpha=0.95,
        max_iter=max_iter
    )

    error_value = abs(sa_result.fun - true_optimum_value) if true_optimum_value is not None else "None"

    results['simulated_annealing'] = {
        'x': sa_result.x,
        'fun': sa_result.fun,
        'nfev': sa_result.nfev + sa_result.njev,
        'nit': sa_result.nit,
        'error_value': error_value,
    }

    print(f"\nOptimization results (method: simulated_annealing):")
    print(f"Found value: {sa_result.fun:.6f}")
    print(f"Number of iterations (nit): {sa_result.nit}")
    print(f"Total function evaluations: {sa_result.nfev + sa_result.njev}")
    print(f"Error from optimum (function value): {error_value:.6f}")

    scipy_methods = ['BFGS', 'SR1', 'Nelder-Mead']

    for method in scipy_methods:
        if method == 'SR1':
            result = minimize(
                objective_func,
                initial_point,
                method='trust-constr',
                options={'maxiter': max_iter},
            )
        else:
            result = minimize(
                objective_func,
                initial_point,
                method=method,
                options={'maxiter': max_iter}
            )

        error_value = abs(result.fun - true_optimum_value) if true_optimum_value is not None else "None"

        results[method] = {
            'x': result.x,
            'fun': result.fun,
            'nfev': result.nfev,
            'nit': result.nit,
            'error_value': error_value,
        }

        print(f"\nOptimization results (method: {method}):")
        print(f"Found value: {result.fun:.6f}")
        print(f"Number of iterations (nit): {result.nit}")
        print(f"Total function evaluations: {result.nfev}")
        print(f"Error from optimum (function value): {error_value:.6f}")

    return results


if __name__ == "__main__":
    print("\n=== Rosenbrock ===")
    run_bench(
        rosenbrock,
        initial_point=np.array([-135.0, 232.0]),
        true_optimum_value=0.0,
    )

    print("\n=== Himmelblau ===")
    run_bench(
        himmelblau,
        initial_point=np.array([-533.0, 313.0]),
        true_optimum_value=0.0,
    )

    print("\n=== Rastrigin ===")
    run_bench(
        rastrigin,
        initial_point=np.array([239.0, -323.0]),
        true_optimum_value=0.0,
    )

    print("\n=== California Housing ===")
    X, y = load_california(scale=True)
    n_features = X.shape[1]
    obj_func = partial(california_objective, X=X, y=y)
    initial_point = np.zeros(n_features + 1)
    run_bench(
        obj_func,
        initial_point=initial_point,
        true_optimum_value=0.5240
    )



=== Rosenbrock ===

Optimization results (method: simulated_annealing):
Found value: 225.928039
Number of iterations (nit): 224000
Total function evaluations: 448001
Error from optimum (function value): 225.928039

Optimization results (method: BFGS):
Found value: 0.000000
Number of iterations (nit): 458
Total function evaluations: 1800
Error from optimum (function value): 0.000000

Optimization results (method: SR1):
Found value: 0.000000
Number of iterations (nit): 269
Total function evaluations: 807
Error from optimum (function value): 0.000000

Optimization results (method: Nelder-Mead):
Found value: 0.000000
Number of iterations (nit): 376
Total function evaluations: 699
Error from optimum (function value): 0.000000

=== Himmelblau ===

Optimization results (method: simulated_annealing):
Found value: 0.000001
Number of iterations (nit): 213000
Total function evaluations: 426001
Error from optimum (function value): 0.000001

Optimization results (method: BFGS):
Found value: 0.0000